In [49]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# if print results in 'cuda' means GPU can be used to process parallel processes

block_size = 8
batch_size = 4
max_iters = 50000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [50]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# All the chars in the text
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [51]:
#Tokenizer

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [52]:
#Train and evaluate results parameters

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[ 1, 62, 67,  1, 69, 58, 71, 69],
        [47, 62, 79, 54, 71, 57,  9,  1],
        [72, 23,  1, 55, 74, 73,  1, 68],
        [68, 76, 67,  1, 74, 67, 73, 62]], device='cuda:0')
targets:
tensor([[62, 67,  1, 69, 58, 71, 69, 65],
        [62, 79, 54, 71, 57,  9,  1, 54],
        [23,  1, 55, 74, 73,  1, 68, 59],
        [76, 67,  1, 74, 67, 73, 62, 65]], device='cuda:0')


In [53]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [54]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Ll5ol(iFKCwhR?_'7ZJvsHp.k0*dZqk;6tT(NtKWe,4U_ q'E)h,(w:]")1J5fp8IrZ5?MiA?d5_iLV)R?do? zazIIM*sLMx &S3[.VKvdyTOe
At6:To;k;z4FEt:KkY;zS xPdMlEn5?YEJ5Atd"n1j
_z;&yi
,o'Po﻿2Nf57PKQ"AwaQWiFB;k9" Boa78Pt6,G6DbVr.V&[D
3vDw6(]*fWa.DUaN5hJ1q(PKG -RN37i1l_:S﻿7s&hij ;2﻿iF3IFZ&eyROtp*(adaLY6EbceZ9!VX0f'gD,Pt0-dIFM2W78g-,2cLd5o0"mBp?4U﻿JHv:pD9PrT0CKvOcLY?uPt60L "-?.l21IMRW-hiPKKav)wOcdgxu&t.:V4QQOQ
386qu9y:"udkw](IvmRfs5_)-A4[gH_&"mOiLBO0Gy
L;D2_HkPjG6?hmBWnMR2v)h5NOrQ574xPVIm*dXJl&Fnvy,4flHN5gEtnD'g*fs4﻿Xig


In [55]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.771, val loss: 4.774
step: 250, train loss: 4.693, val loss: 4.734
step: 500, train loss: 4.645, val loss: 4.650
step: 750, train loss: 4.561, val loss: 4.583
step: 1000, train loss: 4.508, val loss: 4.526
step: 1250, train loss: 4.445, val loss: 4.447
step: 1500, train loss: 4.410, val loss: 4.413
step: 1750, train loss: 4.344, val loss: 4.366
step: 2000, train loss: 4.296, val loss: 4.303
step: 2250, train loss: 4.230, val loss: 4.244
step: 2500, train loss: 4.188, val loss: 4.201
step: 2750, train loss: 4.147, val loss: 4.148
step: 3000, train loss: 4.104, val loss: 4.101
step: 3250, train loss: 4.057, val loss: 4.086
step: 3500, train loss: 3.978, val loss: 4.002
step: 3750, train loss: 3.971, val loss: 3.974
step: 4000, train loss: 3.933, val loss: 3.925
step: 4250, train loss: 3.877, val loss: 3.898
step: 4500, train loss: 3.803, val loss: 3.831
step: 4750, train loss: 3.806, val loss: 3.813
step: 5000, train loss: 3.737, val loss: 3.759
step: 5250, train l

In [56]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"T ais imed thisk!"M)p arot m wad sopreaughu use mothed gexig!29HE h "

oy gers art.

f d g. t atr,  am,


whe l, thyllengan; thq8B'Dop bafo fe tesio aleve the

basty, aiclesothe. b k?(6U(Homeaiffurs S.'OUKEThy ow, ar.
r, t, utist boasthesp,  thealuer herd.


satarppub;
ck in Dongrcangly verrd th s Mud ththoo
"Bube avene flinds ITHethe be," weand
y-APAM oulyld truteris and, schingh asthe reatheblons
 s bl s thoo, We BRACanghe acol tocetof ber or, thigot tswama Tve the to ladeadrrs.
"A, uckethe B
